# Twitter Analyser
David Norrish, November 2017

Call Twitter API to retrieve tweets for a user-defined search query, then performs network and sentiment analysis.

In [6]:
from twitter_search import search

3


usage: ipykernel_launcher.py [-h] [-l LANG] [-r {recent,popular,mixed}]
                             [-c COUNT] [-u YYYY-MM-DD] [-s SINCE_ID]
                             [-m MAX_ID] [-i {true,false}] [-f FILE] [-e]
                             query
ipykernel_launcher.py: error: the following arguments are required: query


SystemExit: 2

c:\Users\Noosh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import bz2
import json
import networkx as nx

# Import and inspect a BZ2 document of tweets in JSON format

tweet_file = bz2.BZ2File('./tweets/marriageEquality.txt.bz2', mode='r')
jobj = json.loads(tweet_file.readline())

# Example of first tweet information
# print(json.dumps(jobj, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
# Parse the JSON tweet object and extract information:
# user/id, retweeted user id
user_id = jobj[u'user'][u'id']
if isinstance(user_id, dict):
    user_id = user_id.values()[0]
user_id = int(user_id) #user_id is u1

print "User id: '%s'" % user_id
if jobj.has_key(u'retweeted_status'):
    retweet_user_id = jobj[u'retweeted_status'][u'user'][u'id']
    if isinstance(retweet_user_id, dict):
        retweet_user_id = retweet_user_id.values()[0]
    retweet_user_id = int(retweet_user_id) #retweet_user_id is u2
    
    print "Retweeted user: '%s'" % retweet_user_id
else:
    print "Not retweeted"